---
#      Lab 3: Frequency Domain Filtering & Hybrid Images
##      Introduction to Computer Vision — ESIN, UIR - Ilias TOUGUI             
###     Full Name: 
###		ID:
---

---

## Instructions
- Read the **theory PDF** before starting this notebook.
- Cells marked with ** YOUR TASK** require you to write code.
- Cells marked with ** REPORT QUESTION** require a written answer in your report.
- Run all cells **in order** from top to bottom.
- Submit this notebook with **all outputs visible**.

---


In [ ]:
# ── CELL: Setup ──────────────────────────────────────────────────
# Run this cell first. All libraries and helper functions are defined here.

import cv2
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import time
import urllib.request

# ─── Image loading helpers ───────────────────────────────────────
def download_img(url, filename):
    urllib.request.urlretrieve(url, filename)

def load_gray(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Could not load '{filename}'. Check the path.")
    return img

def load_color(filename):
    img = cv2.imread(filename)
    if img is None:
        raise FileNotFoundError(f"Could not load '{filename}'. Check the path.")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ─── Display helpers ─────────────────────────────────────────────
def show_image(title, img, cmap='gray'):
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

def show_two_images(title1, img1, title2, img2, cmap1='gray', cmap2='gray'):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(img1, cmap=cmap1)
    axes[0].set_title(title1)
    axes[0].axis('off')
    axes[1].imshow(img2, cmap=cmap2)
    axes[1].set_title(title2)
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()

print(" Setup complete.")


---
## Part 1: Convolution Theorem in Practice — The Speed Race

In the theory PDF, we showed that spatial convolution has complexity
$O(N^2 K^2)$ while FFT-based convolution has complexity $O(N^2 \log N)$.

In this part, you will **measure this difference experimentally** using
`scipy.signal`, which provides both methods from the same library —
making the comparison perfectly fair.

- `scipy.signal.convolve2d` → pure spatial sliding-window convolution
- `scipy.signal.fftconvolve` → FFT-based convolution

---


In [ ]:
# ── CELL: Part 1 — Load image and build kernel ───────────────────

# Download a high-resolution image
download_img('https://cdn.britannica.com/96/178196-050-87B51DDC/Rubiks-Cube-toy.jpg?w=800&h=600&c=crop', 'Rubik_cube.png')

img_color = load_color('Rubik_cube.png')
img_gray = load_gray('Rubik_cube.png')

# Build a large Gaussian kernel
k_size = 51
sigma  = 10
kernel_1d = cv2.getGaussianKernel(k_size, sigma)
kernel_2d = kernel_1d @ kernel_1d.T

# Normalize kernel for display only
kernel_vis = cv2.normalize(kernel_2d, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

show_two_images(
    "Original Image", img_color,
    f"Gaussian Kernel ({k_size}×{k_size})", kernel_vis
)
print(f"Gray Image shape  : {img_gray.shape}")
print(f"Color Image shape  : {img_color.shape}")
print(f"Kernel shape : {kernel_2d.shape}")
print(f"Kernel sum   : {kernel_2d.sum():.6f}")


In [ ]:
# ── CELL: Part 1 — Method A: Spatial Convolution ─────────────────

start = time.time()
spatial_result = scipy.signal.convolve2d(img_gray, kernel_2d, mode='same')
spatial_time   = time.time() - start

print(f"Method A — Spatial Convolution : {spatial_time:.4f} seconds")
show_image("Method A: Spatial Result", spatial_result)


In [ ]:
# ── CELL: Part 1 — Method B: FFT Convolution ─────────────────────

start = time.time()
fft_result = scipy.signal.fftconvolve(img_gray, kernel_2d, mode='same')
fft_time   = time.time() - start

print(f"Method B — FFT Convolution     : {fft_time:.4f} seconds")
show_image("Method B: FFT Result", np.abs(fft_result))


In [ ]:
# ── CELL: Part 1 — Summary ───────────────────────────────────────

print("=" * 45)
print(f"  Spatial time : {spatial_time:.4f} s")
print(f"  FFT time     : {fft_time:.4f} s")
if fft_time > 0:
    print(f"  Speedup      : {spatial_time / fft_time:.1f}×  faster with FFT")
print("=" * 45)


---
###  Report Questions — Part 1

**Q1.** Record the two times printed above. Which method was faster
for a $51 \times 51$ kernel, and by approximately what factor?

**Q2.** Change `k_size` to `3` (and `sigma` to `1`) in the Cells above and rerun them. Which method is now faster for a tiny
$3 \times 3$ kernel? Why does this make sense given the overhead of
computing an FFT?

**Q3.** What theorem guarantees that
spatial convolution and FFT-based convolution must produce the same
output?

---


---
## Part 2: Filtering in the Frequency Domain

We will now filter images **directly in the frequency domain** by
multiplying their spectrum by a mask. We study two types of filters:

1. **Ideal filters** — sharp circular mask (easy to understand, causes ringing)
2. **Gaussian filters** — smooth taper (better quality, no ringing)

---


In [ ]:
# ── CELL: Part 2.1 — Compute and visualize the FFT spectrum ──────

# Compute 2D FFT and shift zero-frequency to center
F       = np.fft.fft2(img_gray)
F_shift = np.fft.fftshift(F)

# Log-magnitude spectrum for visualization
magnitude_spectrum = 20 * np.log(np.abs(F_shift) + 1)

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title("Original Image")
axes[0].axis('off')

im = axes[1].imshow(magnitude_spectrum, cmap='hot')
axes[1].set_title("Magnitude Spectrum (log scale, hot colormap)")
axes[1].axis('off')
fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

print("Center of spectrum = low frequencies (smooth regions)")
print("Outer edges        = high frequencies (sharp edges, textures)")


In [ ]:
# ── CELL: Part 2.2 — Ideal Low-Pass and High-Pass Filters ────────

def ideal_lowpass(img, radius=30, cmap_spec='hot'):
    """
    Ideal low-pass filter with spectrum overlay.
    Keeps a circle of radius r in center, zeros everything outside.
    ⚠ Causes ringing (Gibbs Phenomenon).
    """
    img_float = img.astype(np.float32) / 255.0
    F         = np.fft.fft2(img_float)
    F_shift   = np.fft.fftshift(F)
    mag_spec  = 20 * np.log(np.abs(F_shift) + 1)

    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2

    # Circular mask
    mask = np.zeros((rows, cols), np.uint8)
    cv2.circle(mask, (ccol, crow), radius, 1, thickness=-1)

    # Apply and invert
    img_back = np.abs(np.fft.ifft2(np.fft.ifftshift(F_shift * mask)))
    img_back = np.clip(img_back, 0, 1)

    # Overlay: shade REMOVED region (outside circle) in black
    overlay       = np.zeros((rows, cols, 4), dtype=np.float32)
    overlay[..., 3] = (1 - mask).astype(np.float32) * 0.75

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title("Input Image")
    axes[0].axis('off')

    im = axes[1].imshow(mag_spec, cmap=cmap_spec)
    axes[1].imshow(overlay)
    axes[1].set_title(f"Spectrum — Ideal LP (shaded = removed, r={radius})")
    axes[1].axis('off')
    fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)

    axes[2].imshow(img_back, cmap='gray', vmin=0, vmax=1)
    axes[2].set_title(f"Ideal LP Output (r={radius})\n⚠ Notice ringing near edges")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()
    return img_back


def ideal_highpass(img, radius=30, cmap_spec='hot'):
    """
    Ideal high-pass filter with spectrum overlay.
    Blocks center circle, keeps everything outside.
    ⚠ Causes ringing (Gibbs Phenomenon).
    """
    img_float = img.astype(np.float32) / 255.0
    F         = np.fft.fft2(img_float)
    F_shift   = np.fft.fftshift(F)
    mag_spec  = 20 * np.log(np.abs(F_shift) + 1)

    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2

    mask_lp = np.zeros((rows, cols), np.uint8)
    cv2.circle(mask_lp, (ccol, crow), radius, 1, thickness=-1)
    mask_hp = 1 - mask_lp

    img_back = np.abs(np.fft.ifft2(np.fft.ifftshift(F_shift * mask_hp)))

    # Overlay: shade BLOCKED center in black
    overlay         = np.zeros((rows, cols, 4), dtype=np.float32)
    overlay[..., 3] = mask_lp.astype(np.float32) * 0.75

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title("Input Image")
    axes[0].axis('off')

    im = axes[1].imshow(mag_spec, cmap=cmap_spec)
    axes[1].imshow(overlay)
    axes[1].set_title(f"Spectrum — Ideal HP (shaded = blocked, r={radius})")
    axes[1].axis('off')
    fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)

    axes[2].imshow(np.clip(img_back + 0.3, 0, 1), cmap='gray', vmin=0, vmax=1)
    axes[2].set_title(f"Ideal HP Output (r={radius})\n⚠ Notice ringing near edges")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()
    return img_back

In [ ]:
# ── Run ideal filters ─────────────────────────────────────────────
ilp = ideal_lowpass(img_gray,  radius=30)
ihp = ideal_highpass(img_gray, radius=30)

In [ ]:
# ── CELL: Part 2.3 — Gaussian Low-Pass and High-Pass Filters ─────

def _gaussian_kernel(sigma):
    """Build a normalized 2D Gaussian kernel."""
    size = 8 * sigma + 1
    if size % 2 == 0:
        size += 1
    k1d = cv2.getGaussianKernel(size, sigma)
    return k1d @ k1d.T

def _gaussian_freq_overlay(sigma, rows, cols):
    """Gaussian frequency response (normalized), centered, for overlay."""
    kernel  = _gaussian_kernel(sigma)
    padded  = np.zeros((rows, cols), dtype=np.float32)
    kh, kw  = kernel.shape
    sh, sw  = (rows - kh) // 2, (cols - kw) // 2
    padded[sh:sh+kh, sw:sw+kw] = kernel
    G = np.abs(np.fft.fftshift(np.fft.fft2(padded)))
    return G / G.max()

def gaussian_lowpass(img, sigma=10, cmap_spec='hot'):
    """
    Gaussian low-pass filter with spectrum overlay.
    ✓ No ringing artifacts.
    """
    img_float = img.astype(np.float32) / 255.0
    rows, cols = img.shape

    filtered  = np.clip(
        scipy.signal.fftconvolve(img_float, _gaussian_kernel(sigma), mode='same'),
        0, 1
    )

    F_shift  = np.fft.fftshift(np.fft.fft2(img_float))
    mag_spec = 20 * np.log(np.abs(F_shift) + 1)
    G_over   = _gaussian_freq_overlay(sigma, rows, cols)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title("Input Image")
    axes[0].axis('off')

    im = axes[1].imshow(mag_spec, cmap=cmap_spec)
    axes[1].imshow(G_over, cmap='hot', alpha=0.45)
    axes[1].set_title(f"Spectrum — Gaussian LP (σ={sigma})\n"
                      "bright center = kept  |  fades out = attenuated")
    axes[1].axis('off')
    fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)

    axes[2].imshow(filtered, cmap='gray', vmin=0, vmax=1)
    axes[2].set_title(f"Gaussian LP Output (σ={sigma})\n✓ No ringing")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()
    return filtered


def gaussian_highpass(img, sigma=10, cmap_spec='hot'):
    """
    Gaussian high-pass filter = image minus Gaussian LP.
    ✓ No ringing artifacts.
    """
    img_float = img.astype(np.float32) / 255.0
    rows, cols = img.shape

    lp        = scipy.signal.fftconvolve(img_float, _gaussian_kernel(sigma), mode='same')
    hp        = img_float - lp
    hp_display = np.clip(hp + 0.5, 0, 1)

    F_shift  = np.fft.fftshift(np.fft.fft2(img_float))
    mag_spec = 20 * np.log(np.abs(F_shift) + 1)
    HP_over  = 1.0 - _gaussian_freq_overlay(sigma, rows, cols)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title("Input Image")
    axes[0].axis('off')

    im = axes[1].imshow(mag_spec, cmap=cmap_spec)
    axes[1].imshow(HP_over, cmap='hot', alpha=0.45)
    axes[1].set_title(f"Spectrum — Gaussian HP (σ={sigma})\n"
                      "dark center = blocked  |  bright outer = kept")
    axes[1].axis('off')
    fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)

    axes[2].imshow(hp_display, cmap='gray', vmin=0, vmax=1)
    axes[2].set_title(f"Gaussian HP Output (σ={sigma})\n✓ No ringing")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()
    return hp

In [ ]:
# ── Run Gaussian filters ──────────────────────────────────────────
glp = gaussian_lowpass(img_gray,  sigma=10)
ghp = gaussian_highpass(img_gray, sigma=10)

---
### Report Questions — Part 2

**Q1.** Run `ideal_lowpass` with `radius = 10`, `radius = 50`, and
`radius = 150`. Describe what happens to the image as radius increases.
Complete the sentence:
> *"As the radius increases, the output becomes ______ because we allow
> ______ frequencies to pass through."*

**Q2.** Look carefully at the output of `ideal_lowpass`. Can you see
ripple patterns near sharp edges? What is this phenomenon called,
and why does the ideal circular mask cause it?
*(Hint: think about what the circular mask looks like in the spatial domain.)*

**Q3.** Run both `ideal_lowpass(img_gray, radius=30)` and
`gaussian_lowpass(img_gray, sigma=10)`. Compare the outputs:
which one shows ringing? Which looks more natural? Why?

**Q4.** The Gaussian high-pass is defined as $\text{HP}(f) = f - \text{LP}(f)$.
In your own words, why does subtracting the blurred version from the
original give you the edges?

---


---
### YOUR TASK — Part 2.5: Explore Filter Parameters on Your Own Images

Upload **three images** of your choice:
- One **textured** (e.g. grass, fabric, fur, brick wall)
- One **smooth** (e.g. sky, calm water, plain wall, portrait)
- One **mixed features** (e.g. a car, a cityscape, a person in a landscape)

You will systematically test **several combinations** of:
- **Ideal filter radius** → `radius ∈ {10, 30, 80}`
- **Gaussian filter sigma** → `sigma ∈ {3, 10, 25}`

on all three images and observe how the filter strength interacts
with the image content.

---


In [ ]:
# Download a high-resolution image
# Textured image (grass)
download_img('https://images.stockcake.com/public/1/9/f/19f91cd0-4c24-4355-b675-a88a602617cc_large/dewy-morning-grass-stockcake.jpg', 'grass.jpg')
grass_color = load_color('grass.jpg')
grass_gray = load_gray('grass.jpg')

In [ ]:
# Textured image (grass)
# Ideal vs Gaussian Low Pass Filtering
# Compare radius in (10, 50, 100) 10 vs sigma in (10, 3, 1)
# ...

In [ ]:
# Smooth image (sky)
download_img('https://cdn.theatlantic.com/thumbor/yzr779tnJFIst6qTNe7iWiBKYa4=/845x470:2635x1477/976x549/media/img/mt/2016/03/RTX283V4/original.jpg', 'sky.jpg')
sky_color = load_color('sky.jpg')
sky_gray = load_gray('sky.jpg')

In [ ]:
# Smooth image (sky)
# Ideal vs Gaussian Low Pass Filtering
# Compare radius in (10, 50, 100) 10 vs sigma in (10, 3, 1)
# ...

In [ ]:
# Mixed (Car)
download_img('https://i.pinimg.com/474x/c7/0c/89/c70c89e6afd8b6d826b63b0d64b5b5d2.jpg', 'ek3.jpg')
ek3_color = load_color('ek3.jpg')
ek3_gray = load_gray('ek3.jpg')

In [ ]:
# Smooth image (Car)
# Ideal vs Gaussian Low Pass Filtering
# Compare radius in (10, 50, 100) 10 vs sigma in (10, 3, 1)
# ...

In [ ]:
# Smooth image (You own Image)
# Download and load it

In [ ]:
# Smooth image (You own Image)
# Ideal vs Gaussian Low Pass Filtering
# Compare radius in (10, 50, 100) 10 vs sigma in (10, 3, 1)
# ...

---
## Part 3: Hybrid Images

A hybrid image combines the **low-frequency content** of one image
with the **high-frequency content** of another:

$$H = \text{LP}_{\sigma_1}(A) + \alpha \cdot \text{HP}_{\sigma_2}(B)$$

- **Up close**: high frequencies dominate → you see image $B$
- **From far away**: only low frequencies remain → you see image $A$

The following functions are provided. They work on **grayscale images**.
Your challenge in Task 3.3 is to extend them to **color**.

---


In [ ]:
# ── CELL: Part 3 — Provided functions (grayscale) ────────────────

def match_histogram(source, reference):
    """Match mean and std of source to reference. Both float32 in [0,1]."""
    src_mean, src_std = source.mean(), source.std()
    ref_mean, ref_std = reference.mean(), reference.std()
    if src_std < 1e-6:
        return source
    return np.clip((source - src_mean) * (ref_std / src_std) + ref_mean, 0, 1)


def lowpass_filter(img, sigma=10):
    """Gaussian LP filter. img: 2D float32 in [0,1]. Returns filtered 2D array."""
    size = 8 * sigma + 1
    if size % 2 == 0:
        size += 1
    k1d = cv2.getGaussianKernel(size, sigma)
    return scipy.signal.fftconvolve(img, k1d @ k1d.T, mode='same')


def highpass_filter(img, sigma=10):
    """Gaussian HP filter = img - LP(img). Returns array in [-0.5, 0.5]."""
    return img - lowpass_filter(img, sigma)

---
### YOUR CHALLENGE — Part 3.1: Color Hybrid Images

Your task is to complete the **color version**
below, which operates on **RGB images of shape `(H, W, 3)`**.

The structure is already provided for you — you only need to fill in
the **three `TODO` blocks** inside the `for c in range(3)` loop.

---

#### Key things to notice
- The `for c in range(3)` loop runs **three times** — once for R, G, B.
- Each iteration extracts one 2D slice `img[:, :, c]` and processes it
  independently using the same grayscale functions from Part 2.
- `np.stack(..., axis=-1)` at the end reassembles the three channels
  back into a `(H, W, 3)` color image.
- You do **not** need to modify anything outside the loop.

---


In [ ]:
def hybrid_image(low_freq_img, high_freq_img, sigma_low=6, sigma_high=3, alpha=1.0, match_hist=True):
    """
    Create a COLOR hybrid image.
    low_freq_img:  uint8 RGB image (H, W, 3) — seen from FAR
    high_freq_img: uint8 RGB image (H, W, 3) — seen UP CLOSE
    sigma_low:     blur strength for far image (higher = blurrier)
    sigma_high:    edge sharpness for close image (smaller = sharper)
    alpha:         weight of high-frequency component [0.5 to 1.5]
    match_hist:    normalize brightness between images per channel
    """
    # 0. Auto-resize to same dimensions
    h, w = low_freq_img.shape[:2]
    high_freq_img = cv2.resize(high_freq_img, (w, h))

    # 1. Convert to float32 in [0, 1]
    img1 = low_freq_img.astype(np.float32) / 255.0
    img2 = high_freq_img.astype(np.float32) / 255.0

    # 2. Process each channel independently
    lp_channels      = []
    hp_channels      = []
    hybrid_channels  = []

    for c in range(3):  # R, G, B
        ch1 = img1[:, :, c]  # one channel from low-freq image
        ch2 = img2[:, :, c]  # one channel from high-freq image

        # Match brightness/contrast per channel
        if match_hist:
            # --- TODO ---
            # ch2 = ...

        # Apply filters
        # --- TODO ---
        # lp = ...
        # hp = ...

        # Blend
        # --- TODO ---
        # hybrid_ch = ... 

        lp_channels.append(np.clip(lp,           0.0, 1.0))
        hp_channels.append(np.clip(hp + 0.5,     0.0, 1.0))
        hybrid_channels.append(np.clip(hybrid_ch, 0.0, 1.0))

    # 3. Stack channels back into (H, W, 3)
    lp_color     = np.stack(lp_channels,     axis=-1)
    hp_color     = np.stack(hp_channels,     axis=-1)
    hybrid_color = np.stack(hybrid_channels, axis=-1)

    return hybrid_color, lp_color, hp_color



In [ ]:
def visualize_hybrid(hybrid_img, num=5):
    """
    Shows the COLOR hybrid at decreasing sizes side by side.
    Simulates stepping back from the image.
    """
    # Convert float [0,1] → uint8 [0,255] for cv2.resize
    img = (np.clip(hybrid_img, 0, 1) * 255).astype(np.uint8)

    gap = 5
    max_height = img.shape[0]

    # Pre-compute all resized images first (avoids off-by-one pixel errors)
    resized_list = [cv2.resize(img, (0, 0), fx=0.5**i, fy=0.5**i) for i in range(num)]

    # Build canvas with 3 channels (H, W, 3) — white background
    total_width = sum(r.shape[1] for r in resized_list) + gap * (num - 1)
    canvas = np.ones((max_height, total_width, 3), dtype=np.uint8) * 255

    current_x = 0
    for r in resized_list:
        h, w = r.shape[:2]
        # Place each image at the bottom of the canvas
        canvas[max_height - h:, current_x:current_x + w, :] = r
        current_x += w + gap

    # Display — no cmap needed for color images
    plt.figure(figsize=(18, 5))
    plt.imshow(canvas)
    plt.title("Hybrid image (color) — close up (left) → far away (right)")
    plt.axis('off')
    plt.show()


In [ ]:
# Download sample aligned dog/cat images
download_img('https://raw.githubusercontent.com/rhthomas/hybrid-images/master/data/dog.bmp', 'dog.jpg')
download_img('https://raw.githubusercontent.com/rhthomas/hybrid-images/master/data/cat.bmp', 'cat.jpg')

dog = cv2.resize(load_color('dog.jpg'), (512, 512))
cat = cv2.resize(load_color('cat.jpg'), (512, 512))

# Build hybrid: dog from far, cat up close
hybrid, lp_dog, hp_cat = hybrid_image(
    low_freq_img=dog,
    high_freq_img=cat,
    sigma_low=6,
    sigma_high=3,
    alpha=1.0
)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(lp_dog, cmap='gray', vmin=0, vmax=1)
axes[0].set_title("Low-Pass Dog (seen from far)")
axes[0].axis('off')
axes[1].imshow(hp_cat, cmap='gray', vmin=0, vmax=1)
axes[1].set_title("High-Pass Cat (seen up close)")
axes[1].axis('off')
axes[2].imshow(hybrid,  cmap='gray', vmin=0, vmax=1)
axes[2].set_title("Hybrid Image (Grayscale)")
axes[2].axis('off')
plt.show()

visualize_hybrid(hybrid)

---
### YOUR TASK — Part 3.2: Test with Your Own Image Pair

Upload **two images** of your choice. For best results:
- Both images should have **similar framing** (e.g. both faces)
- Key features should be **roughly aligned**
- Both should have **similar overall brightness**

---


In [ ]:
# Test Your combination here

---
###  Report Questions — Part 3

**Q1.** Describe what you see when you **zoom in** vs **zoom out**
on the hybrid image. Use the terms *high frequency* and
*low frequency* in your explanation.

**Q2.** Try `sigma_low = 3` vs `sigma_low = 15` with the sample
dog/cat images. How does it affect what you see from a distance?
Why?

**Q3.** Swap the two images (put cat as low-pass and dog as
high-pass) and re-run. Does the illusion still work?
Which direction produces a more convincing hybrid, and why?

**Q4. (Bonus)** In `visualize_hybrid`, at approximately which scale
(1/2, 1/4, 1/8 ...) does the far-away image start to clearly
dominate? Does changing `sigma_low` shift this crossover point?

---

## Submission Checklist

Before submitting, make sure:
- [ ] All cells are executed and outputs are visible
- [ ] You tested with your own image pair (Part 3.2)
- [ ] Your report (max 2 pages PDF) answers all questions above
- [ ] Your report includes a screenshot of your best hybrid image

---
